In [1]:
import requests
import time
import random

### 어제 만들었던 download 함수 

In [2]:
def download(method="get", url="", headers={}, params={}, retries=3):
    try:
        resp = requests.request(method, 
                                url, 
                                headers=headers,
                                params=params if method=="get" else {},
                                data = params if method=="post" else {}
                               )
        resp.raise_for_status()
        
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print("Retries:",retries)
            time.sleep(random.randint(1,5))
            return download(method, url, headers, params, retries-1)
            
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            print(resp.url)

    return resp

## 새로운 테스트 사이트
- http://pythonscraping.com/pages/files/form.html
- post 방식
- ID, PW 로그인하는 형식
- ID, PW는 아무거나 입력해도 된다.

In [3]:
# 실제 접속하는 url은 아래이지만,
url = "http://pythonscraping.com/pages/files/form.html"
# 개발자도구를 열고 로그인을 해본 후, Request header를 보면 url이 다음과 같다.
# 보이는 url과 실제 request를 받는 url이 다르다. 숨겨놓은 것.
url = "http://pythonscraping.com/pages/files/processing.php"

# network 탭에서 header정보를 보면 맨 아래에 Form Data 가 있다. get 방식에서의  Query String Parameter와 같은 역할이다. 
# post 방식의 parmeter이다.
# get 방식과 달리, url에 파라미터가 보이지 않는다. => 보안
params = {
    "firstname" : "testKey",
    "lastname" : "testValue"
}

# post 방식을 사용함.
resp = download("post", url, params=params)

In [4]:
# 정상적으로 resopnse 받음.

resp.status_code, resp.request.body, resp.url, resp.text

(200,
 'firstname=testKey&lastname=testValue',
 'http://pythonscraping.com/pages/files/processing.php',
 'Hello there, testKey testValue!')

## 교보문고 사이트에서 실습
- http://www.kyobobook.co.kr/
- post 방식 검색

In [5]:
url = "http://www.kyobobook.co.kr/search/SearchCommonMain.jsp"

params = {
    "vPstrCategory": "TOT",
    "vPstrKeyWord": "python",
    "vPplace": "top",
    "vPorderClick": "LET",
    "searchCategory": "TOT",
    "searchKeyword": "python",
    "eventurlFlag": 0,
    "eventurlDelFlag": 1
}

resp = download("post", url, params=params)

In [6]:
# 제대로 response 받아왔는지 체크

resp.status_code, resp.reason, resp.url

(200, 'OK', 'http://www.kyobobook.co.kr/search/SearchCommonMain.jsp')

In [7]:
resp.text

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml">\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n<!-- header -->\n \n\n\n\n\n\n\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\n\n\n<head>\n\t<title>python -  인터넷교보문고</title>\n<META http-equiv="Pragma" content="nocache">\n<meta http-equiv="Expires" content="0"/>\n<META http-equiv="Cache-Control" content="no-cache">\n<META http-equiv=\'Content-Type\' content=\'text/html; charset=euc-kr\'>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge,chrome=1">\n<!--<LINK href="/common/Style.css" rel="stylesheet" type="text/css"/>-->\n<link rel="shortcut icon" href="/newimages/apps/b2c/kyobo.ICO"/>\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/default_ink.css" />\n<link rel="stylesheet" type="text/css" href="http://image.kyobobook.co.kr/ink/css/search.css" />\n<!--\n<link rel="stylesh

---

# Cookies

#### 쿠키정보 확인하는 방법
- 개발자도구(단축키 f12) -> Application 탭(화면 상단) -> Storage/Cookies(화면 좌측)

## 쿠키 테스트 사이트
- http://pythonscraping.com/pages/cookies/login.html
- 로그인 테스트 사이트
- ID는 아무거나 상관없음, PW는 password일때만 로그인 성공한다.

### 1. 하던 방법대로, download 함수를 이용해서 접근해보자
- login이 안되는게 정상임

In [8]:
url = "http://pythonscraping.com/pages/cookies/welcome.php"
params = {
    "username": "id", 
    "password": "password"
}

resp = download("post", url, params=params)

In [11]:
# 정상적으로 resp를 받기는 했다.

resp.status_code, resp.url, resp.request.body

(200,
 'http://pythonscraping.com/pages/cookies/welcome.php',
 'username=id&password=password')

In [13]:
# 내용을 보면 login 실패라는 문구가 떴다.
# 브라우저가 없는 상태이기 때문에, cookies 값이 없어서 login이 안되는 것이 정상이다.

print(resp.text)


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


### 2. session을 이용해서 로그인하기
- 로그인 된다.
- resquests.request => resquests.get, requests.post
    - requests.request는 .get과 .post랑 사실 같다.
    - 이 녀석들은 모두 cookies 값을 저장하지 못한다.
    - 이 녀석들은 모두 Session 정보를 저장하지 못한다.
    - session을 활용하려면 resquests.Session을 써야한다.

In [14]:
# session을 만들자

session = requests.Session()

In [38]:
resp = session.post(url, data=params)

# 정상적으로 response 받아온다.
resp.status_code, resp.request.body, resp.text

(200,
 'username=id&password=password',
 '\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>')

In [43]:
# session 객체에 cookie 값이 저장되었다.

session.cookies.items()

[('loggedin', '1'), ('username', 'id')]

In [40]:
# type이 CookieJar이다.
# 귀여워...

type(session.cookies)

requests.cookies.RequestsCookieJar

### 3. resquest 메서드에 cookies 인자를 줬을때
- 로그인 된다.

In [44]:
# cookie 값을 같이 보내면 로그인이 된다.
# 이런 방법으로, 로그인한 상태로 웹사이트를 돌아다닐 수 있게 된다.
# 최초 로그인해서 cookie 값이 생성된 후에 이 방법을 쓸 수 있다.

resp = requests.post(url, cookies=session.cookies)
resp.text

'\n<h2>Welcome to the Website!</h2>\nYou have logged in successfully! <br><a href="profile.php">Check out your profile!</a>'

## 종합

1. Download 함수(DIY) 썼을 때 => requests.request와 결과 같음, 로그인 x
2. Session 을 썼을 때=> cookie 값을 저장하므로, 로그인 O
3. requests.post with Jar(cookies) 썼을 때 => cookie 값 있으므로, 로그인 O

# 쿠팡에서 cookies로 로그인하기 실습

In [82]:
%%writefile coupang.json

# 본인 ID와 PW를 json 파일로 저장합니다.
# 개인정보 보호를 위해서!

{
    "id":"아이디",
    "pw":"비밀번호"
}

Overwriting coupang.json


In [69]:
import json

In [87]:
with open("coupang.json", encoding="utf-8") as fp:
    account = json.load(fp)

In [137]:
# 일부러 ID/PW를 틀리면 request headers 정보를 볼 수 있습니다. 

url = "https://login.coupang.com/login/loginProcess.pang"
params = {
    "email": "Own Email",
    "password": "Own PW",
    "rememberMe": "false",
    "token": "null",
    "captchaAnswer": "null",
    "returnUrl": "http%253A%252F%252Fwww.coupang.com"
}

In [138]:
session = requests.Session()
resp = session.post(url, data=params)

In [139]:
resp.status_code, resp.reason, resp.url

(200, 'OK', 'https://login.coupang.com/login/loginProcess.pang')

In [140]:
# 로그인이 안되네요....
# 수업시간에 결국 하지 못하고 넘어갔습니다...

resp.json()

{'success': False,
 'resultTypeCode': 'FAIL',
 'message': '이메일 또는 비밀번호를 다시 확인하세요. 쿠팡에 등록되지 않은 이메일이거나, 이메일 또는 비밀번호를 잘못 입력하셨습니다.',
 'data': None}

In [132]:
session.cookies.items()

[('CLKEY',
  'b292djlUT0JFNFBBdVVrMk43VENwNFF4YzRHamhBQ3lvNGlXVkxwYytGd3gwVjNVSEc4SEVObU9ncHQ4bnlIcmlPYy90RER4ZVVvUWtyOE00UksxdHhwdFBPVTdZVWhBc2hBNWhqVmRXd289'),
 ('PCID', '15675949893625180697104'),
 ('sid', '6098011e9d3442c4b4557fa2ad53c3c64636cf9e'),
 ('PCID', '10383128424864303431733')]